### Ranking recommendation algorithms using Multi-Armed Bandit 

Multi-Armed Bandit has many application methods; one of the main ones is a ranking system built using this algorithm.

In this case, we can imagine each recommendation algorithm as an unknown distribution that returns a value depending on the algorithm's accuracy.


I decided to use Slate Bandit Framework with top k = 1, which means we will show recommendations from only one recomender system simultaneously.

In the case of top_k > 1, we could decide that the combination of models is our arm, but this could increase the number of arms exponentially, so be careful. 


There are a large number of methods you can use to determine the accuracy of a recommender system. I will use the following one.

First of all, we will train our models on data up to a certain date, and for simplicity, the models will not be retrained until the end of the simulation.

After that, we will select a random user and conduct testing on him.

Next, we will go through orders NOT used for model training in chronological order and determine which arms recommended this facility.

For those arms that could guess the next institution, we will give $(star rating - 2.5) ** 3 / sqrt(rank_n)$,
where star_rating is the rating given by the guest, and rank_n is the order of this institution in the recommendation system. For other systems, we put 0.

*We can change the rating system as needed.

After a guest visits each restaurant, we will change the order of our recommendation systems according to bandit choice. However, since our system is static, changing the order will not affect which restaurants the guest visits.

So, to solve this problem, we will also calculate a score for each Bandit. The Bandit will receive points only if the user visits a facility recommended by his arm.

The Bandit with the biggest score is the best one.



In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import sys
sys.path.append('../../')
import random
import warnings

from src.utils import read_json_df
from src.models.median_baseline import MedianBaselineModel
from src.models.page_rank import PageRankModel
import pandas as pd
from bandit import GreadyBandit,ThompsonBandit
warnings.filterwarnings("ignore")

In [3]:
review_df = read_json_df("../../data/yelp_dataset/yelp_academic_dataset_review.json")
user_df = read_json_df("../../data/yelp_dataset/yelp_academic_dataset_user.json")
business_df = read_json_df("../../data/yelp_dataset/yelp_academic_dataset_business.json")

In [4]:
review_df['date'] = pd.to_datetime(review_df['date'])
review_df = review_df[review_df.date < pd.to_datetime('2008-01-01')]
user_df = user_df[user_df.user_id.isin(review_df.user_id.unique())]
business_df = business_df[business_df.business_id.isin(review_df.business_id.unique())]

In [5]:
review_df.date.min()

Timestamp('2005-02-16 03:23:22')

In [6]:
date = pd.to_datetime('2007-01-01')
user_ids = list(review_df.user_id.unique())
N = 100

In [14]:
# Train model only ones for all bandits, to be sure that the models are similar
models = [MedianBaselineModel(),PageRankModel()]
bandit_gready = GreadyBandit(models,business_df,review_df,user_df,N)
bandit_gready.train_models(date)

START FITTING A MODEL
MODEL IS FITTED
START FITTING A MODEL
MODEL IS FITTED


In [15]:
ranks = bandit_gready.simulate_data(user_ids,date)

15363it [00:13, 1125.16it/s]


In [16]:
bandit_thompson = ThompsonBandit(models,business_df,review_df,user_df,N)

In [17]:
ranks = bandit_thompson.simulate_data(user_ids,date)

15363it [00:14, 1071.93it/s]


In [18]:
bandit_thompson.bandit_score,bandit_gready.bandit_score

(652.3738886456875, 649.9371484429024)

In our case, we can see that the Thompson model performed better because it scored more points.

However, as both algorithms are using random it's better to use Hyphothesis testing before tell that one bandit is better.


In [19]:
bandit_thompson.models_score

[555.4222882650068, 652.3738886456875]